In [1]:
from alpdesign.utils import *
from alpdesign.mlp import *
from jax_unirep import get_reps
import alpdesign
import numpy as np
import jax_unirep
import haiku as hk
import jax
import jax.numpy as jnp
import functools

In [2]:
def hamming(seq1, seq2): # seq1 and seq2 are of same length
    seqlist1 = list(seq1)
    seqlist2 = list(seq2)
    hamming_distance = 0.
    for i in range(len(seqlist1)):
        if seqlist1[i] == seqlist2[i]:
            continue
        hamming_distance += 1.
    return hamming_distance / len(seqlist1)

In [3]:
key = jax.random.PRNGKey(0)
batch_size=16
target_seq = 'GIGAVLKVLTTGLPALISWIKRKRQQ'
oh_vec = encode_seq(list(target_seq))
oh_unirep = seq2useq(oh_vec)
target_rep = differentiable_jax_unirep(oh_unirep)
#print(target_rep.shape)
seqs = ['GIGAVLKVLKAGLPALIVTLKRKIVQ',
       'PPGATLKKHTTGSVALISWIWARIQQ',
       'GIGAVLKVLTTGLKTLISAAKRKRAA',
       'HAPPVLKVLTTGLAPPLVWIKRKRTH',
       'GIGAVLUIHKLSSVAAWRPPKRKRQQ',
       'PTWIIFLKAQWEQHSNLTNMRTFPEV',
        'TISHFVCNHDICAWIKDMQAMQIKMC',
        'CESWLWKRLFDGHADRWRSMPDYPIW',
        'YLVENPLMFPLVAAFIHQWTRQISWH',
        'QTEERLEAQISIYYIGAWSHYKVTDE']
reps = get_reps(seqs)[0]

labels = []
for seq in seqs:
    labels.append(hamming(target_seq, seq))
labels = jnp.array(labels)

In [4]:
key = jax.random.PRNGKey(0)
c = alpdesign.EnsembleBlockConfig()
forward_fxn, full_forward_fxn = alpdesign.build_model(c)
full_forward_t = hk.without_apply_rng(hk.transform(full_forward_fxn))
params, losses = alpdesign.ensemble_train(
    key, full_forward_t, c, reps, labels, epochs=5, learning_rate=0.01)
forward_t = hk.without_apply_rng(hk.transform(forward_fxn))
forward = functools.partial(forward_t.apply, params)

# e2e is a haiku func
def e2e(logits):
    s = alpdesign.SeqpropBlock()(logits)
    us = alpdesign.seq2useq(s)
    u = alpdesign.differentiable_jax_unirep(us)
    return forward(u)
e2e_t = hk.transform(e2e)
init_logits = jax.random.normal(key, shape=((26, 20)))
e2e_params = e2e_t.init(key, init_logits)

def e2e_fxn(x, key):
    e2e_params, logits = x
    yhat = e2e_t.apply(e2e_params, key, logits)
    return yhat
x = alpdesign.bayes_opt(key, e2e_fxn, labels, init_x=(e2e_params, init_logits), iter_num=10)

0
1
2
3
4
5
6
7
8
9


In [5]:
print(x)

(FlatMap({
  'seqprop': FlatMap({
               'b': DeviceArray(-0.02555459, dtype=float32),
               'r': DeviceArray(1.0262499, dtype=float32),
             }),
}), DeviceArray([[ 7.36458600e-01, -5.48070312e-01, -1.03321743e+00,
              -9.13318336e-01,  9.49477732e-01,  1.80428600e+00,
               2.06541210e-01,  3.73605460e-01,  3.27033371e-01,
              -2.48704538e-01, -1.15040863e+00,  2.31316105e-01,
               1.00427818e+00,  3.73366565e-01, -1.92696559e+00,
               1.49402905e+00, -1.19397378e+00, -1.43170249e+00,
               1.12819210e-01, -4.92835402e-01],
             [-9.97294068e-01,  1.24625456e+00,  3.69796276e-01,
              -2.26057887e+00, -1.52431977e+00,  1.10461271e+00,
               3.48834753e-01,  1.17481387e+00, -6.89574420e-01,
              -1.84043169e+00,  2.99874663e-01, -3.56299073e-01,
               1.00192142e+00, -4.35048431e-01,  2.50580549e-01,
              -2.89858460e+00, -4.51895207e-01, -6.91504776e-

               2.54388124e-01,  5.08339405e-01]], dtype=float32))
